In [1]:
import os, sys, codecs, time, datetime
import fileinput
import os.path
import pickle
from io import StringIO
from subprocess import call


from docopt import docopt
from propsde.applications.viz_tree import DepTreeVisualizer
%load_ext autoreload
%autoreload 2
import propsde.applications.run as run
import sys
if sys.version_info[0] >= 3:
    unicode = str
    
import spacy
from spacy_conll import ConllFormatter



### CorZu 2.0

In [2]:
import os
import urllib.request
import urllib.parse
import stanza
from stanza.utils.conll import CoNLL
import subprocess

def _get_spacy_nlp():
    spacy.prefer_gpu()
    nlp = spacy.load('de_core_news_md')
    conllformatter = ConllFormatter(nlp)
    nlp.add_pipe(conllformatter, after='parser')
    return nlp

def _get_stanza_nlp():
    nlp = stanza.Pipeline('de')
    return nlp

def _parse_dependencies_to_conll_file_stanza(text, nlp, conll_name):
    doc = nlp(text).to_dict() 
    conll = CoNLL.convert_dict(doc)
    conll_str = ""
    for sentence in conll:
        for word in sentence:
            for attr in word:
                conll_str += attr + "\t"
            conll_str += "\n"
            conll_str = conll_str.replace("\t\n", "\n")
    try:
        file = open(conll_name,'w')
        file.write(conll_str)
        return 0
    except Exception as ex:
        print(ex)
        return 1
    
def _write_to_file(text, file_name):
    try:
        file = open(file_name,'w')
        file.write(text)
        return 0
    except Exception as ex:
        print(ex)
        return 1
    
def _parse_dependencies_parzu_http(text):
    params = {
        "text": text,
        "format": "conll"
    }
    args = urllib.parse.urlencode(params).encode("utf-8")
    req = "http://localhost:5003/parse/?"+args.decode("utf-8")
    try:
        f = urllib.request.urlopen(req)
        return f.read().decode("utf-8")
    except Exception as ex:
        print(ex)
        return None 

def _parse_dependencies_to_conll_file_parzu_http(text, conll_name):
    params = {
        "text": text,
        "format": "conll"
    }
    args = urllib.parse.urlencode(params).encode("utf-8")
    req = "http://localhost:5003/parse/?"+args.decode("utf-8")
    try:
        f = urllib.request.urlopen(req)
        _write_to_file(f, conll_name)
        return 0
    except Exception as ex:
        print(ex)
        return 1
    
def _parse_dependencies_to_conll_file_parzu(text, conll_name):
    cmd = "echo '" + text + "' | docker run -i rsennrich/parzu /ParZu/parzu"
    process = subprocess.Popen(['docker','run', "-i", "rsennrich/parzu", "/ParZu/parzu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    data = process.communicate()
    print(data[1].decode('utf-8'))
    
def _parse_dependencies_to_conll_file_spacy(text, nlp, conll_name):
    doc = nlp(text)
    conll = doc._.conll_str
    print(conll)
    conll = conll.replace("\n\n", "\n")
    _write_to_file(conll, conll_name)

def _create_markables(conll_name, markable_name):
    cmd = "python ext/CorZu_v2.0/extract_mables_from_parzu.py " + conll_name + " > " + markable_name
    err = os.system(cmd)
    return err

def execute_corzu(text, nlp=None, conll_name="output.conll", markable_name="markable", output="corzu_export"):
    if nlp:
        print("Using Stanza")
        #_parse_dependencies_to_conll_file_spacy(text, nlp, conll_name)
        _parse_dependencies_to_conll_file_stanza(text, nlp, conll_name)
    else:
        print("Using ParZu")
        _parse_dependencies_to_conll_file_parzu(text, conll_name)
    err = _create_markables(conll_name, markable_name)
    if err == 0:
        cmd = "python ext/CorZu_v2.0/corzu.py " + markable_name + " " + conll_name + " " + output    
        err = os.system(cmd)
        return err
    else:
        print("Markables Error:", err)


### Read Tuebadz

In [3]:
from collections import defaultdict

def read_tuebadz_per_sentence(path):
    i = 0
    raw_data = {}
    gold_standard = {}
    sentences = defaultdict(list)
    for line in open(path,'r').readlines():
        li=line.strip()
        if li.startswith("#"):
            li = li.split()
            if li[1] == 'newdoc':
                new_doc = li[4]
                gold_standard[new_doc] = ""
            if li[1] == 'sent_id':
                sent_id = li[3]
            if li[1] == 'text':
                sentences[new_doc].append(line[9:].rstrip(" \n"))
        else:
            gold_standard[new_doc] += line
        #if i == 1000:
        #    break
        #i += 1
    sentences[new_doc].append(line[9:].rstrip(" \n"))
    return sentences, gold_standard
            

raw_sentences, gold_standard = read_tuebadz_per_sentence("tuebadz-11.0-conll-u-v2.txt")

In [4]:
nlp = _get_spacy_nlp()

In [48]:
def _parse_parzu():
    i = 0
    conll = ""
    for new_doc in raw_sentences:
        text = raw_sentences[new_doc]
        conll += "\n#begin\n"
        for s in text:
            try:
                conll += _parse_dependencies_parzu_http(s)
            except Exception as ex:
                print(new_doc)
                print(ex)


In [51]:
import os
output = "corzu_export"
markables = "markables"
text = open("examples.txt", "r").read()

cmd = "python ext/CorZu_v2.0/extract_mables_from_parzu.py " + conll + " > " + markables
err = os.system(cmd)
print(err)

256


In [13]:
#print(raw_sentences[0]["text"])
execute_corzu(raw_sentences[0]["text"])

Using ParZu
HTTP Error 408: REQUEST TIMEOUT


0

In [12]:
execute_corzu("Peter geht spazieren, er mag Pommes")

Using ParZu


0

In [4]:
nlp = _get_stanza_nlp()


2020-06-30 12:25:50 INFO: Loading these models for language: de (German):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |
| depparse  | gsd     |
| ner       | conll03 |

2020-06-30 12:25:50 INFO: Use device: gpu
2020-06-30 12:25:50 INFO: Loading: tokenize
2020-06-30 12:25:52 INFO: Loading: mwt
2020-06-30 12:25:52 INFO: Loading: pos
2020-06-30 12:25:53 INFO: Loading: lemma
2020-06-30 12:25:53 INFO: Loading: depparse
2020-06-30 12:25:54 INFO: Loading: ner
2020-06-30 12:25:54 INFO: Done loading processors!


Using Stanza


0

In [7]:
execute_corzu("Peter geht spazieren, er mag Pommes", nlp, conll_name="output_spacy")

Using Stanza


0

1 Das der PRON PDS Case=Nom|Gender=Neut|Number=Sing|PronType=Dem 4 nsubj _ start_char=0|end_char=3 
2 ist sein AUX VAFIN Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin 4 cop _ start_char=4|end_char=7 
3 ein ein DET ART Case=Nom|Definite=Ind|Gender=Masc|Number=Sing|PronType=Art 4 det _ start_char=8|end_char=11 
4 Test Test NOUN NN Case=Nom|Gender=Masc|Number=Sing 0 root _ start_char=12|end_char=16 
5 . . PUNCT $. _ 4 punct _ start_char=16|end_char=17 
1 Er er PRON PPER Case=Nom|Gender=Masc|Number=Sing|Person=3|PronType=Prs 3 nsubj _ start_char=18|end_char=20 
2 ist sein AUX VAFIN Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin 3 cop _ start_char=21|end_char=24 
3 Gut Gut NOUN NN Case=Nom|Gender=Neut|Number=Sing 0 root _ start_char=25|end_char=28 
4 . . PUNCT $. _ 3 punct _ start_char=28|end_char=29 

